Required Packages:
- pandas  : 1.5.3
- openpyxl: 3.1.5
- xlrd    : 2.0.1

# 📚 | Import Libraries 

In [ ]:
import config as cfg
import pandas as pd
import numpy as np
import openpyxl
import xlrd
import os
import zipfile
from io import BytesIO
from datetime import datetime

from library.path_utils import get_file_path, to_absolute_path

print("pandas  :", pd.__version__)
print("openpyxl:", openpyxl.__version__)
print("xlrd    :", xlrd.__version__)

# ⚙️ | Settings

In [ ]:
RAW_PXPN_DIR = "./raw_data/PXPN"

enroll_file_name = "pxpn_enroll_info"
zip_file_name = "pixelpanic_raw_data.zip"
output_folder_name = "./_tmp/PXPN"

# Code

In [ ]:
enroll_path = get_file_path(RAW_PXPN_DIR, f"{enroll_file_name}.xlsx")
zip_path = get_file_path(RAW_PXPN_DIR, f"{zip_file_name}")
output_folder = to_absolute_path(output_folder_name)
os.makedirs(output_folder, exist_ok=True)
csv_path = get_file_path(RAW_PXPN_DIR, f"{enroll_file_name}.csv")

# 엑셀 읽고 csv로 저장
df = pd.read_excel(enroll_path)
df.to_csv(csv_path, index=False)

enroll_path = get_file_path(RAW_PXPN_DIR, f"{enroll_file_name}.csv")

In [ ]:
import zipfile
import os
from io import BytesIO
import pandas as pd

outer_zip_path = zip_path

# 1) 바깥 ZIP 열기
with zipfile.ZipFile(outer_zip_path, 'r') as outer_zip:
    # 2) ActiveData 폴더 안의 inner-zip 경로 수집
    active_data_zips = {}
    for member in outer_zip.namelist():
        # 경로에 'ActiveData/' 포함하고, '_ActiveData.zip' 으로 끝나는 파일만 선택
        if 'ActiveData/' in member and member.endswith('_ActiveData.zip'):
            # 파일명에서 patient_code(예: 'ABC123') 추출
            base = os.path.basename(member)                # e.g. 'ABC123_ActiveData.zip'
            patient_code = base.replace('_ActiveData.zip', '')
            active_data_zips[patient_code] = member

    # 3) 모은 경로들을 순회하며 내부 ZIP 열기
    for patient_code, inner_path in active_data_zips.items():
        # outer_zip.read()로 바이트 읽기 → BytesIO로 감싸기
        inner_bytes = BytesIO(outer_zip.read(inner_path))
        with zipfile.ZipFile(inner_bytes, 'r') as inner_zip:
            # 원하는 CSV 파일명
            survey_csv = f'{patient_code}_SurveyResponse.csv'
            if survey_csv not in inner_zip.namelist():
                print(f'[경고] {patient_code} 내부에 {survey_csv} 없음')
                continue

            # CSV 열어서 DataFrame으로 읽기
            with inner_zip.open(survey_csv) as f:
                df = pd.read_csv(f)
                print(f'{patient_code}: 읽은 행 수 =', len(df))
                # TODO: df 처리 로직 추가

In [ ]:
import zipfile
import os
from io import BytesIO
import pandas as pd


                 


# 1) 설문 리스트
top_5 = [
    '특성 불안 설문', '한국형 회복탄력성 지수', '한국어판 아침형-저녁형 설문지',
    '한글판 생물학적 리듬 평가 설문지', '유년기 외상 척도', '한국형 기분장애 설문지',
    '광장공포 인지 설문지', '알바니 공황-공포 질문지', '신체감각 설문지',
    '한글판 범불안 장애', '한국어판 우울증 선별도구'
]
result = pd.DataFrame(columns=['patient_code', '날짜'])
# 2) 결과 DataFrame 초기화
import zipfile
import os
from io import BytesIO
import pandas as pd

outer_zip_path = zip_path

# 1) 바깥 ZIP 열기
with zipfile.ZipFile(outer_zip_path, 'r') as outer_zip:
    # 2) ActiveData 폴더 안의 inner-zip 경로 수집
    active_data_zips = {}
    for member in outer_zip.namelist():
        # 경로에 'ActiveData/' 포함하고, '_ActiveData.zip' 으로 끝나는 파일만 선택
        if 'ActiveData/' in member and member.endswith('_ActiveData.zip'):
            # 파일명에서 patient_code(예: 'ABC123') 추출
            base = os.path.basename(member)                # e.g. 'ABC123_ActiveData.zip'
            patient_code = base.replace('_ActiveData.zip', '')
            active_data_zips[patient_code] = member

    # 3) 모은 경로들을 순회하며 내부 ZIP 열기
    for patient_code, inner_path in active_data_zips.items():
        # outer_zip.read()로 바이트 읽기 → BytesIO로 감싸기
        inner_bytes = BytesIO(outer_zip.read(inner_path))
        with zipfile.ZipFile(inner_bytes, 'r') as inner_zip:
            # 원하는 CSV 파일명
            survey_csv = f'{patient_code}_SurveyResponse.csv'
            if survey_csv not in inner_zip.namelist():
                print(f'[경고] {patient_code} 내부에 {survey_csv} 없음')
                continue

            # CSV 열어서 DataFrame으로 읽기
            with inner_zip.open(survey_csv) as f:
                df = pd.read_csv(f)
                # TODO: df 처리 로직 추가
            # 작성일 컬럼에서 날짜만 뽑기
            date_value = pd.to_datetime(df['작성일'].iloc[0]).date()

            # 결과 DataFrame에 (patient_code, 날짜) 행 추가
            mask = (result['patient_code'] == patient_code) & (result['날짜'] == date_value)
            if not mask.any():
                result = pd.concat([
                    result,
                    pd.DataFrame([{'patient_code': patient_code, '날짜': date_value}])
                ], ignore_index=True)

            # 5) 설문별 점수 추출 및 결과에 삽입
            for survey in top_5:
                sub = df[df['설문명'] == survey].reset_index(drop=True)
                if sub.empty:
                    continue

                # 실제 점수(real_score) 리스트
                scores = []
                for _, row in sub.iterrows():
                    if row.get('역채점인 경우 역채점 점수', '-') != '-':
                        scores.append(float(row['역채점인 경우 역채점 점수']))
                    else:
                        v = row.get('점수')
                        scores.append(float(v) if pd.notna(v) else '***')

                # 설문명 → 컬럼 접두사 매핑
                prefix_map = {
                    '특성 불안 설문': 'STAI_X2',
                    '한국형 회복탄력성 지수': 'KRQ',
                    '한국어판 아침형-저녁형 설문지': 'CSM',
                    '한글판 생물학적 리듬 평가 설문지': 'BRIAN',
                    '한국형 기분장애 설문지': 'MDQ',
                    '광장공포 인지 설문지': 'ACQ',
                    '신체감각 설문지': 'BSQ',
                    '한글판 범불안 장애': 'GAD',
                    '한국어판 우울증 선별도구': 'PHQ',
                    # 유년기 외상 척도 → CTQ, 알바니 공황-공포 → APPQ (주제별)
                    '유년기 외상 척도': 'CTQ',
                    '알바니 공황-공포 질문지': 'APPQ'
                }
                prefix = prefix_map[survey]

                # 주제별 분리 필요한 설문
                if survey in ['유년기 외상 척도', '알바니 공황-공포 질문지']:
                    sub['real_score'] = scores
                    topics = sorted(sub['주제'].dropna().unique())
                    for ti, topic in enumerate(topics, start=1):
                        tdf = sub[sub['주제'] == topic].reset_index(drop=True)
                        for qi, sc in enumerate(tdf['real_score'], start=1):
                            col = f"{prefix}-{ti}-{qi}"
                            result.loc[
                                (result['patient_code'] == patient_code) &
                                (result['날짜'] == date_value),
                                col
                            ] = sc
                else:
                    for idx, sc in enumerate(scores, start=1):
                        col = f"{prefix}-{idx}"
                        result.loc[
                            (result['patient_code'] == patient_code) &
                            (result['날짜'] == date_value),
                            col
                        ] = sc

# 6) 컬럼 순서 재배열
cols = ['patient_code', '날짜'] + [c for c in result.columns if c not in ['patient_code', '날짜']]
result = result[cols]

# 7) 불필요 컬럼 삭제 (예: 범위를 벗어난 MDQ-14,15 등)
result = result.drop(columns=['MDQ-14', 'MDQ-15', 'PHQ-10'], errors='ignore')

# 8) 저장
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "questionnaire.csv")
result.to_csv(output_path, index=False)

In [ ]:
# 사용할 접두어 목록
prefixes = ["PHQ", "STAI_X2", "CSM", "CTQ-1", "CTQ-2", "CTQ-3", "CTQ-4", "CTQ-5", "KRQ", "MDQ", "ACQ", "APPQ-1", "APPQ-2", "APPQ-3", "BSQ", "GAD", "BRIAN"]
# 결과를 저장할 데이터프레임
aggregated_df = result[['patient_code', '날짜']].copy()

for prefix in prefixes:
    # 해당 접두어로 시작하는 컬럼 찾기
    matched_cols = [col for col in result.columns if col.startswith(prefix)]
    
    # 값 합산해서 새로운 컬럼으로 추가
    aggregated_df[f'{prefix.rstrip("-")}'] = result[matched_cols].apply(pd.to_numeric, errors='coerce').sum(axis=1)

display(aggregated_df.head(3))


In [ ]:
# 엑셀 파일 경로
file_path = enroll_path

# 엑셀 시트 읽기
df = pd.read_csv(file_path)

# 컬럼 이름 변경
df = df.rename(columns={'회원코드': 'patient_code', '2. 성별': 'gender'})

# 필요한 컬럼만 선택
df = df[['patient_code', '연구시작일', '연구종료일', 'gender']]

# 날짜 형식으로 변환
df['연구시작일'] = pd.to_datetime(df['연구시작일'], errors='coerce')
df['연구종료일'] = pd.to_datetime(df['연구종료일'], errors='coerce')

# 각 환자에 대해 날짜 생성
expanded_rows = []
for _, row in df.iterrows():
    if pd.notnull(row['연구시작일']) and pd.notnull(row['연구종료일']):
        date_range = pd.date_range(start=row['연구시작일'], end=row['연구종료일'])
        for date in date_range:
            expanded_rows.append({'patient_code': row['patient_code'], '날짜': date, 'gender': row['gender']})

# 결과 데이터프레임 생성
expanded_df = pd.DataFrame(expanded_rows)
expanded_df = expanded_df.replace({'남': '0', '여': '1'})
print(expanded_df)


In [ ]:
# 엑셀 파일 경로
file_path = enroll_path

# CSV 읽기
df = pd.read_csv(file_path)

# 컬럼 이름 변경 (생년월일 컬럼도 추가)
df = df.rename(columns={
    '회원코드': 'patient_code',
    '2. 성별': 'gender'  # 생년월일 컬럼 이름이 실제 다르면 이 부분 수정 필요
})

# 필요한 컬럼만 선택
df = df[['patient_code', '연구시작일', '연구종료일', 'gender', '3. 생년월일']]

# 날짜 형식으로 변환
df['연구시작일'] = pd.to_datetime(df['연구시작일'], errors='coerce')
df['연구종료일'] = pd.to_datetime(df['연구종료일'], errors='coerce')
df['3. 생년월일'] = pd.to_datetime(df['3. 생년월일'], errors='coerce')


# 각 환자에 대해 날짜 생성
expanded_rows = []
for _, row in df.iterrows():
    if pd.notnull(row['연구시작일']) and pd.notnull(row['연구종료일']):
        date_range = pd.date_range(start=row['연구시작일'], end=row['연구종료일'])
        for date in date_range:
            expanded_rows.append({
                'patient_code': row['patient_code'],
                '날짜': date,
                'gender': '0' if row['gender'] == '남' else '1'
            })

# 결과 데이터프레임 생성
expanded_df = pd.DataFrame(expanded_rows)

# 출력
print(expanded_df)

In [ ]:
expanded_df['날짜']    = pd.to_datetime(expanded_df['날짜'])
aggregated_df['날짜'] = pd.to_datetime(aggregated_df['날짜'])
expanded_answer = pd.merge(expanded_df, aggregated_df, on=['patient_code', '날짜'], how='outer')
expanded_answer = expanded_answer.rename(columns={'patient_code': 'ID', '날짜': 'date', 'GAD': 'GAD_7', 'CTQ-1': 'CTQ_1', 'CTQ-2': 'CTQ_2', 'CTQ-3': 'CTQ_3', 'CTQ-4': 'CTQ_4', 'CTQ-5': 'CTQ_5', 'APPQ-1': 'APPQ_1', 'APPQ-2': 'APPQ_2', 'APPQ-3': 'APPQ_3', 'PHQ': 'PHQ_9'})
output_path = os.path.join(output_folder, "questionnaire_test.csv")
expanded_answer.to_csv(output_path, index=False)

In [ ]:
# 0. 날짜 형식 통일
expanded_answer['date'] = pd.to_datetime(expanded_answer['date'])
# 0) 결과 DataFrame 준비
PXPN_panic_dates = pd.DataFrame(columns=['ID', 'date'])


with zipfile.ZipFile(zip_path, 'r') as outer_zip:
    for member in outer_zip.namelist():
        # ➤ 끝이 _ActiveData.zip 이고, 경로 어딘가에 ActiveData/ 포함, 숨김파일 제외
        if (
            member.endswith('_ActiveData.zip') and
            'ActiveData/' in member and
            not any(x in member for x in ['__MACOSX', '/._', '.DS_Store'])
        ):
            # patient ID 추출
            pid = os.path.basename(member).replace('_ActiveData.zip', '')

            # 내부 ZIP 열기
            buf = BytesIO(outer_zip.read(member))
            buf.seek(0)
            if not zipfile.is_zipfile(buf):
                print(f"❌ {member} 는 ZIP이 아닙니다.")
                continue
            buf.seek(0)

            with zipfile.ZipFile(buf, 'r') as inner_zip:
                # Panic.csv 찾기
                panic_files = [f for f in inner_zip.namelist() if f.endswith('Panic.csv')]
                if not panic_files:
                    print(f"⚠️ {pid}: Panic.csv 없음")
                    continue

                # (여러 개 있을 수 있으니 모두 처리)
                for panic_fname in panic_files:
                    with inner_zip.open(panic_fname) as f:
                        df_panic = pd.read_csv(f)
                        if '작성일' not in df_panic.columns:
                            print(f"⚠️ {pid}: 작성일 컬럼 없음 in {panic_fname}")
                            continue

                        # 날짜별로 한 행씩 추가
                        for d in pd.to_datetime(df_panic['작성일'], errors='coerce').dt.date.unique():
                            PXPN_panic_dates = pd.concat([
                                PXPN_panic_dates,
                                pd.DataFrame({'ID': [pid], 'date': [d]})
                            ], ignore_index=True)

# 2) panic 값, 포맷 정리
PXPN_panic_dates['panic'] = 2
PXPN_panic_dates['date']  = pd.to_datetime(PXPN_panic_dates['date'])\
                                .dt.strftime('%Y-%m-%d')

# 3) expanded_answer 쪽도 date 컬럼 문자열 포맷으로
expanded_answer['date'] = pd.to_datetime(expanded_answer['date'], errors='coerce')\
                               .dt.strftime('%Y-%m-%d')

# 3. outer merge
merged = pd.merge(
    PXPN_panic_dates,
    expanded_answer,
    on=['ID', 'date'],
    how='outer'
)

# 4. 우선순위 panic 값 유지
merged = (
    merged
    .sort_values(['ID', 'date', 'panic'], ascending=[True, True, False])
    .drop_duplicates(subset=['ID', 'date'], keep='first')
)

# 5. 전날 panic=1 적용
df = merged.copy()
df['date'] = pd.to_datetime(df['date'])

for _, row in df[df['panic'] == 2].iterrows():
    pid = row['ID']
    curr_date = row['date']
    prev_date = curr_date - pd.Timedelta(days=1)

    # 연속된 2 제거
    while ((df['ID'] == pid) & (df['date'] == prev_date) & (df['panic'] == 2)).any():
        prev_date -= pd.Timedelta(days=1)

    mask = (df['ID'] == pid) & (df['date'] == prev_date)
    if mask.any():
        df.loc[mask & (df['panic'].fillna(0) < 1), 'panic'] = 1

df = df.sort_values(['ID', 'date']).reset_index(drop=True)
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df['panic'] = df['panic'].fillna(0)

# 6. Enroll 병합 (age)
enroll_df = pd.read_csv(enroll_path, encoding='utf-8')
enroll_df = enroll_df.rename(columns={
    '회원코드': 'patient_code',
    '3. 생년월일': 'birthdate',
    '연구종료일': 'end_date'
})
enroll_df['birthdate'] = pd.to_datetime(enroll_df['birthdate'], errors='coerce')
enroll_df['end_date'] = pd.to_datetime(enroll_df['end_date'], errors='coerce')
enroll_df['age'] = enroll_df['end_date'].dt.year - enroll_df['birthdate'].dt.year

age_df = enroll_df[['patient_code', 'age']].drop_duplicates().rename(columns={'patient_code': 'ID'})
final_df = df.merge(age_df, on='ID', how='left')

# 7. 저장
output_path = os.path.join(output_folder, "questionnaire_and_panic_date.csv")
final_df.to_csv(output_path, index=False)

In [ ]:
# 파일 경로
csv_path = output_path
zip_path = zip_path
# 데이터 불러오기
df = pd.read_csv(csv_path)
df = df[df["ID"].str.startswith("PXPN")].copy()
df["date"] = pd.to_datetime(df["date"])

# 컬럼 초기화
for col in ['marriage', 'job', 'alcohol', 'coffee', 'smoking', 'menstruation', 'exercise',
            'smkHx', 'drinkHx', 'suicideHx', 'suicide_need']:
    df[col] = np.nan

with zipfile.ZipFile(zip_path, 'r') as outer_zip:
    all_files = outer_zip.namelist()

    # ActiveData ZIP 파일만 골라서 {pid: 경로} 매핑
    active_data_zips = {
        os.path.basename(f).replace('_ActiveData.zip', ''): f
        for f in all_files
        if f.startswith("pixelpanic_raw_data/ActiveData/") and f.endswith("_ActiveData.zip")
           and "__MACOSX" not in f and "/._" not in f and ".DS_Store" not in f
    }

    for pid in df['ID'].unique():
        pid = str(pid).strip()
        if pid not in active_data_zips:
            continue

        # inner ZIP 바이트로 읽어서 열기
        inner_path = active_data_zips[pid]
        buf = BytesIO(outer_zip.read(inner_path))
        with zipfile.ZipFile(buf, 'r') as inner_zip:

            # Sociodemographic.csv 처리
            soc_file = f"{pid}_Sociodemographic.csv"
            if soc_file in inner_zip.namelist():
                # header=None, index_col=0 로 읽고 transpose
                soc_df = pd.read_csv(inner_zip.open(soc_file), header=None, index_col=0).T
                if '결혼' in soc.columns:
                    df.loc[df['ID'] == pid, 'marriage'] = 1 if soc['결혼'].values[0] == '기혼' else 0
                if '현재 직업 유무' in soc.columns:
                    df.loc[df['ID'] == pid, 'job'] = 1 if soc['현재 직업 유무'].values[0] == 'Y' else 0
                if '과거 흡연 여부' in soc.columns:
                    df.loc[df['ID'] == pid, 'smkHx'] = 1 if soc['과거 흡연 여부'].values[0] == 'Y' else 0
                if '지금까지 음주 여부' in soc.columns:
                    df.loc[df['ID'] == pid, 'drinkHx'] = 1 if soc['지금까지 음주 여부'].values[0] == 'Y' else 0
                if '과거 자살 시도 여부' in soc.columns:
                    df.loc[df['ID'] == pid, 'suicideHx'] = 1 if soc['과거 자살 시도 여부'].values[0] == 'Y' else 0
                if '지난 1달간 자살시도 여부' in soc.columns:
                    df.loc[df['ID'] == pid, 'suicide_need'] = 1 if soc['지난 1달간 자살시도 여부'].values[0] == 'Y' else 0


            # 2. Pattern 처리
            pat_path = f"{pid}_Pattern.csv"
            if pat_path in inner_zip.namelist():
                pat = pd.read_csv(inner_zip.open(pat_path))
                pat['작성일'] = pd.to_datetime(pat['작성일'], errors='coerce')

                for idx, row in df[df["ID"] == pid].iterrows():
                    d = row["date"]
                    today_rows = pat[pat["작성일"] == d]
                    for _, r in today_rows.iterrows():
                        t = r.get('종류', '')
                        st = r.get('세부종류', '')
                        amount = r.get('양', None)  # '양' 컬럼 값
                        # 운동
                        if t == '운동':
                            # 양 값이 있으면 그 값을, 없으면 1 로 디폴트
                            df.at[idx, 'exercise'] = amount if pd.notna(amount) else 1
                        # 카페인
                        if t == '카페인':
                            df.at[idx, 'coffee'] = amount if pd.notna(amount) else 1
                        # 흡연
                        if t == '흡연':
                            df.at[idx, 'smoking'] = amount if pd.notna(amount) else 1
                        # 음주(양이 아닌 단순 여부만 원하면 기존처럼 1로)
                        if t == '음주':
                            df.at[idx, 'alcohol'] = amount if pd.notna(amount) else 1
                        # 생리
                        if t == '생리' and st == '생리중':
                            df.at[idx, 'menstruation'] = 1

output_path = os.path.join(output_folder, "questionnaire_and_panic_dates_and_demo.csv")
df.to_csv(output_path, index=False)

In [ ]:
import os
import zipfile
from io import BytesIO
import pandas as pd
import numpy as np

# --- 설정: 상대 경로 지정 ---
csv_path   = output_path

# 1) 원본 CSV 읽기 및 필터링
df = pd.read_csv(csv_path)
df = df[df['ID'].astype(str).str.startswith('PXPN')].copy()
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# 2) 컬럼 초기화
cols_to_add = ['marriage','job','alcohol','coffee','smoking',
               'menstruation','exercise','smkHx','drinkHx',
               'suicideHx','suicide_need']
for col in cols_to_add:
    df[col] = np.nan

# 3) ZIP 열기 및 내부 ZIP 매핑
with zipfile.ZipFile(zip_path, 'r') as outer_zip:
    all_files = outer_zip.namelist()
    active_data_zips = {
        os.path.basename(f).replace('_ActiveData.zip',''): f
        for f in all_files
        if 'ActiveData/' in f and f.endswith('_ActiveData.zip')
           and '__MACOSX' not in f and '/._' not in f and '.DS_Store' not in f
    }

    # 4) 각 ID별 Sociodemographic & Pattern 처리
    for pid in df['ID'].astype(str).unique():
        inner_path = active_data_zips.get(pid)
        if not inner_path:
            print(f"⚠️ {pid}: ActiveData ZIP 미발견")
            continue

        buf = BytesIO(outer_zip.read(inner_path))
        buf.seek(0)
        with zipfile.ZipFile(buf,'r') as inner_zip:
            names = inner_zip.namelist()

            # Sociodemographic.csv 처리
            soc_file = f"{pid}_Sociodemographic.csv"
            if soc_file in names:
                soc_df = pd.read_csv(inner_zip.open(soc_file), header=None, index_col=0).T
                m = lambda v,p='Y': 1 if str(v).strip()==p else 0
                # 결혼 여부
                if '결혼' in soc_df.columns:
                    df.loc[df['ID']==pid,'marriage'] = m(soc_df['결혼'].iloc[0], '기혼')
                # 현재 직업 유무
                if '현재 직업 유무' in soc_df.columns:
                    df.loc[df['ID']==pid,'job'] = m(soc_df['현재 직업 유무'].iloc[0])
                # 과거 흡연 여부
                if '과거 흡연 여부' in soc_df.columns:
                    df.loc[df['ID']==pid,'smkHx'] = m(soc_df['과거 흡연 여부'].iloc[0])
                # 지금까지 음주 여부
                if '지금까지 음주 여부' in soc_df.columns:
                    df.loc[df['ID']==pid,'drinkHx'] = m(soc_df['지금까지 음주 여부'].iloc[0])
                # 과거 자살 시도 여부
                if '과거 자살 시도 여부' in soc_df.columns:
                    df.loc[df['ID']==pid,'suicideHx'] = m(soc_df['과거 자살 시도 여부'].iloc[0])
                # 지난 1달간 자살시도 여부
                if '지난 1달간 자살시도 여부' in soc_df.columns:
                    df.loc[df['ID']==pid,'suicide_need'] = m(soc_df['지난 1달간 자살시도 여부'].iloc[0])
            else:
                print(f"⚠️ {pid}: {soc_file} 없음. 내부 파일들: {names}")

            # Pattern 처리
            pat_file = f"{pid}_Pattern.csv"
            if pat_file in names:
                pat_df = pd.read_csv(inner_zip.open(pat_file))
                pat_df['작성일'] = pd.to_datetime(pat_df['작성일'], errors='coerce')
                for idx, row in df[df['ID']==pid].iterrows():
                    d = row['date'].date()
                    today = pat_df[pat_df['작성일'].dt.date==d]
                    for _, r in today.iterrows():
                        t, st = r.get('종류',''), r.get('세부종류','')
                        amt = r.get('양', np.nan)
                        if t=='운동': df.at[idx,'exercise']=amt if pd.notna(amt) else 1
                        elif t=='카페인': df.at[idx,'coffee']=amt if pd.notna(amt) else 1
                        elif t=='흡연': df.at[idx,'smoking']=amt if pd.notna(amt) else 1
                        elif t=='음주': df.at[idx,'alcohol']=amt if pd.notna(amt) else 1
                        elif t=='생리' and st=='생리중': df.at[idx,'menstruation']=1
            else:
                print(f"⚠️ {pid}: {pat_file} 없음. 내부 파일들: {names}")

output_path = os.path.join(output_folder, "questionnaire_and_panic_dates_and_demo.csv")
df.to_csv(output_path, index=False)

In [ ]:
# 1. 경로 설정
zip_path = zip_path
processed = df  

# 3. PXPN ID 목록 추출
pxpn_ids = processed[processed['ID'].astype(str).str.startswith('PXPN')]['ID'].unique()

# 4. 감정 관련 컬럼 초기화
emotion_cols = ['positive_feeling', 'negative', 'positive_E', 'negative_E', 'anxiety', 'annoying']
for col in emotion_cols:
    if col not in processed.columns:
        processed[col] = np.nan

# 5. 디버그용 카운터 및 정보
match_count = 0
no_date_match = 0
processed_ids = set()
debug_info = []

# 6. 외부 zip 열기
import zipfile
from io import BytesIO

debug_info = []

# 1) ZIP 열기
with zipfile.ZipFile(zip_path, 'r') as outer_zip:
    all_files = outer_zip.namelist()

    # 2) ActiveData ZIP 매핑 (PID → 내부 경로)
    active_data_zips = {
        os.path.basename(f).replace('_ActiveData.zip', ''): f
        for f in all_files
        if 'ActiveData/' in f and f.endswith('_ActiveData.zip')
           and '__MACOSX' not in f and '/._' not in f and '.DS_Store' not in f
    }

    # 3) 모든 PXPN ID에 대해 반복
    for pid in pxpn_ids:
        pid = str(pid).strip()
        inner_path = active_data_zips.get(pid)
        if not inner_path:
            debug_info.append(f"❌ ID {pid}: ActiveData ZIP 없음")
            continue

        # 4) 내부 ZIP 열기
        buf = BytesIO(outer_zip.read(inner_path))
        buf.seek(0)
        if not zipfile.is_zipfile(buf):
            debug_info.append(f"❌ ID {pid}: 유효한 ZIP 아님")
            continue
        buf.seek(0)

        with zipfile.ZipFile(buf, 'r') as inner_zip:
            # 5) Checkup.csv 경로 확인
            checkup_file = f"{pid}_Checkup.csv"
            if checkup_file not in inner_zip.namelist():
                debug_info.append(f"⚠️ ID {pid}: {checkup_file} 없음")
                continue

            # 6) Checkup CSV 읽기
            with inner_zip.open(checkup_file) as f:
                checkup = pd.read_csv(f)

            # 날짜 타입 변환
            processed_pid = processed[processed['ID'] == pid].copy()
            processed_pid['date'] = pd.to_datetime(processed_pid['date'], errors='coerce')
            checkup['작성일'] = pd.to_datetime(checkup['작성일'], errors='coerce')

            # 감정 카테고리별 처리
            for category in ['기분', '에너지', '불안', '짜증']:
                category_data = checkup[checkup['종류'] == category]

                for _, row in category_data.iterrows():
                    checkup_date = row['작성일']
                    score = row['척도']

                    for idx, proc_row in processed_pid.iterrows():
                        proc_date = proc_row['date']
                        if (
                            proc_date.year == checkup_date.year and
                            proc_date.month == checkup_date.month and
                            proc_date.day == checkup_date.day
                        ):
                            if category == '기분':
                                if score > 0:
                                    processed.at[idx, 'positive_feeling'] = score
                                elif score < 0:
                                    processed.at[idx, 'negative'] = score
                            elif category == '에너지':
                                if score > 0:
                                    processed.at[idx, 'positive_E'] = score
                                elif score < 0:
                                    processed.at[idx, 'negative_E'] = score
                            elif category == '불안':
                                processed.at[idx, 'anxiety'] = score
                            elif category == '짜증':
                                processed.at[idx, 'annoying'] = score

                            match_count += 1
                            processed_ids.add(pid)



# 8. 기분 및 에너지 충돌 조정 (절대값 기준)
mask_mood = processed['positive_feeling'].notna() & processed['negative'].notna()
for idx in processed[mask_mood].index:
    pos = processed.at[idx, 'positive_feeling']
    neg = processed.at[idx, 'negative']
    if abs(pos) > abs(neg):
        processed.at[idx, 'negative'] = 0
    elif abs(pos) < abs(neg):
        processed.at[idx, 'positive_feeling'] = 0
    else:
        processed.at[idx, 'positive_feeling'] = 0  # 동일하면 긍정 제거, 부정 유지

mask_energy = processed['positive_E'].notna() & processed['negative_E'].notna()
for idx in processed[mask_energy].index:
    posE = processed.at[idx, 'positive_E']
    negE = processed.at[idx, 'negative_E']
    if abs(posE) > abs(negE):
        processed.at[idx, 'negative_E'] = 0
    elif abs(posE) < abs(negE):
        processed.at[idx, 'positive_E'] = 0
    else:
        processed.at[idx, 'positive_E'] = 0

# 9. 값이 한쪽만 있을 경우 다른 쪽을 0으로 설정
processed.loc[
    processed['positive_feeling'].notna() & processed['negative'].isna(),
    'negative'
] = 0
processed.loc[
    processed['negative'].notna() & processed['positive_feeling'].isna(),
    'positive_feeling'
] = 0

processed.loc[
    processed['positive_E'].notna() & processed['negative_E'].isna(),
    'negative_E'
] = 0
processed.loc[
    processed['negative_E'].notna() & processed['positive_E'].isna(),
    'positive_E'
] = 0

# 10. 디버그 출력 (최대 20개)
for info in debug_info[:20]:
    print(info)


output_path = os.path.join(output_folder, "questionnaire_panic_demo_mood.csv")
processed.to_csv(output_path, index=False)

In [ ]:
from pandas.api.types import is_numeric_dtype, is_string_dtype, is_datetime64_any_dtype

# 전체 컬럼 순회하며 형식 통일
for col in processed.columns:
    if col == 'ID':
        processed[col] = processed[col].astype(str).str.strip()
    elif col == 'date':
        processed[col] = pd.to_datetime(processed[col], errors='coerce')  # 문자열 포함 시 자동 처리
    elif is_datetime64_any_dtype(processed[col]):
        processed[col] = pd.to_datetime(processed[col], errors='coerce')  # datetime이면 그대로
    elif is_string_dtype(processed[col]):
        processed[col] = processed[col].astype(str).str.strip()  # 문자열이면 정리
    else:
        # 예외적인 경우도 문자열로 통일
        processed[col] = processed[col].astype(str).str.strip()


output_path = os.path.join(output_folder, "processed.csv")
processed.to_csv(output_path, index=False)
print("모든 컬럼 형식 통일 완료 및 저장됨.")

In [ ]:
print(processed.columns)